<a href="https://colab.research.google.com/github/ciepielajan/NLP_Text-Summarization/blob/main/2_0_Text_Summarizaton_Tf_Idf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

2_0_Text Summarizaton_Tf-Idf.ipynb

`pobranie bibliotek`

In [1]:
import pandas as pd
import nltk
import re
import nltk
nltk.download('punkt')
from nltk import sent_tokenize, word_tokenize, PorterStemmer
nltk.download('stopwords')
from nltk.corpus import stopwords
stopWords = set(stopwords.words("english"))
ps = PorterStemmer()
import math

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
!pip -q  install wikipedia    # -q = quite (hide output)
import wikipedia

`surowy tekst`

In [3]:
text_row = wikipedia.page("Machine Learing").content
text_row

'Machine learning (ML) is the study of computer algorithms that improve automatically through experience and by the use of data. It is seen as a part of artificial intelligence. Machine learning algorithms build a model based on sample data, known as "training data", in order to make predictions or decisions without being explicitly programmed to do so. Machine learning algorithms are used in a wide variety of applications, such as email filtering and computer vision, where it is difficult or unfeasible to develop conventional algorithms to perform the needed tasks.\nA subset of machine learning is closely related to computational statistics, which focuses on making predictions using computers; but not all machine learning is statistical learning. The study of mathematical optimization delivers methods, theory and application domains to the field of machine learning. Data mining is a related field of study, focusing on exploratory data analysis through unsupervised learning. In its app

In [4]:
print("format: ", type(text_row))
print("ilość znaków: ", len(text_row))
print("ilość słow: ", len(text_row.split(" ")))
print("ilość zdań: ", len(sent_tokenize(text_row)))

format:  <class 'str'>
ilość znaków:  45879
ilość słow:  6956
ilość zdań:  263


`oczyszczenie tekstu`

> wikipedia formatuje paragrafy wg wzorca:

'=== JAKAŚ TREŚĆ ==='

W pierwszej kolejności wyszukajmy wszystkie paragrafy

In [5]:
wzorzec = r"\=+\s+[\w\s\-]+\=+"
print("znalezionych wystąpień: ", len(re.findall(wzorzec,text_row)))
print(re.findall(wzorzec,text_row))

znalezionych wystąpień:  46
['== Overview ==', '=== Machine learning approaches ===', '== History and relationships to other fields ==', '=== Artificial intelligence ===', '=== Data mining ===', '=== Optimization ===', '=== Generalization ===', '=== Statistics ===', '== Theory ==', '== Approaches ==', '=== Types of learning algorithms ===', '==== Supervised learning ====', '==== Unsupervised learning ====', '==== Semi-supervised learning ====', '==== Reinforcement learning ====', '==== Self learning ====', '==== Feature learning ====', '==== Sparse dictionary learning ====', '==== Anomaly detection ====', '==== Robot learning ====', '==== Association rules ====', '=== Models ===', '==== Artificial neural networks ====', '==== Decision trees ====', '==== Support-vector machines ====', '==== Regression analysis ====', '==== Bayesian networks ====', '==== Genetic algorithms ====', '=== Training models ===', '==== Federated learning ====', '== Applications ==', '== Limitations ==', '=== Bi

In [6]:
text = re.sub(wzorzec, " ",text_row) # usuń paragrafy
text = re.sub(r'\n+', " ",text)  # usuń znaki nowej linii 
text = re.sub(r'\s{2,}', " ",text) # usuń wielokrotności spacji
text = text.strip() # usunięcie pierwszej i ostatniej spacji
text

'Machine learning (ML) is the study of computer algorithms that improve automatically through experience and by the use of data. It is seen as a part of artificial intelligence. Machine learning algorithms build a model based on sample data, known as "training data", in order to make predictions or decisions without being explicitly programmed to do so. Machine learning algorithms are used in a wide variety of applications, such as email filtering and computer vision, where it is difficult or unfeasible to develop conventional algorithms to perform the needed tasks. A subset of machine learning is closely related to computational statistics, which focuses on making predictions using computers; but not all machine learning is statistical learning. The study of mathematical optimization delivers methods, theory and application domains to the field of machine learning. Data mining is a related field of study, focusing on exploratory data analysis through unsupervised learning. In its appl

In [7]:
print("format: ", type(text))
print("ilość znaków: ", len(text))
print("ilość słow: ", len(text.split(" ")))
print("ilość zdań: ", len(sent_tokenize(text)))

format:  <class 'str'>
ilość znaków:  44241
ilość słow:  6603
ilość zdań:  263


`Tf-Idf`

`1 Sentence Tokenize`

In [45]:
# 1 Sentence Tokenize
sentences = sent_tokenize(text)
total_documents = len(sentences)
print(sentences)
print(total_documents)

['Machine learning (ML) is the study of computer algorithms that improve automatically through experience and by the use of data.', 'It is seen as a part of artificial intelligence.', 'Machine learning algorithms build a model based on sample data, known as "training data", in order to make predictions or decisions without being explicitly programmed to do so.', 'Machine learning algorithms are used in a wide variety of applications, such as email filtering and computer vision, where it is difficult or unfeasible to develop conventional algorithms to perform the needed tasks.', 'A subset of machine learning is closely related to computational statistics, which focuses on making predictions using computers; but not all machine learning is statistical learning.', 'The study of mathematical optimization delivers methods, theory and application domains to the field of machine learning.', 'Data mining is a related field of study, focusing on exploratory data analysis through unsupervised le

`2 Create the Frequency matrix of the words in each sentence.`

In [46]:
def _create_frequency_matrix(sentences):
    frequency_matrix = {}
    stopWords = set(stopwords.words("english"))
    ps = PorterStemmer()

    for id, sent in enumerate(sentences):
        freq_table = {}
        words = word_tokenize(sent)
        for word in words:
            word = word.lower()
            word = ps.stem(word)
            if word in stopWords:
                continue

            if word in freq_table:
                freq_table[word] += 1
            else:
                freq_table[word] = 1

        frequency_matrix[id] = freq_table  # frequency_matrix[sent[:15]] = freq_table

    return frequency_matrix

In [47]:
freq_matrix = _create_frequency_matrix(sentences)
print(freq_matrix)

{0: {'machin': 1, 'learn': 1, '(': 1, 'ml': 1, ')': 1, 'studi': 1, 'comput': 1, 'algorithm': 1, 'improv': 1, 'automat': 1, 'experi': 1, 'use': 1, 'data': 1, '.': 1}, 1: {'seen': 1, 'part': 1, 'artifici': 1, 'intellig': 1, '.': 1}, 2: {'machin': 1, 'learn': 1, 'algorithm': 1, 'build': 1, 'model': 1, 'base': 1, 'sampl': 1, 'data': 2, ',': 2, 'known': 1, '``': 1, 'train': 1, "''": 1, 'order': 1, 'make': 1, 'predict': 1, 'decis': 1, 'without': 1, 'explicitli': 1, 'program': 1, '.': 1}, 3: {'machin': 1, 'learn': 1, 'algorithm': 2, 'use': 1, 'wide': 1, 'varieti': 1, 'applic': 1, ',': 2, 'email': 1, 'filter': 1, 'comput': 1, 'vision': 1, 'difficult': 1, 'unfeas': 1, 'develop': 1, 'convent': 1, 'perform': 1, 'need': 1, 'task': 1, '.': 1}, 4: {'subset': 1, 'machin': 2, 'learn': 3, 'close': 1, 'relat': 1, 'comput': 2, 'statist': 2, ',': 1, 'focus': 1, 'make': 1, 'predict': 1, 'use': 1, ';': 1, '.': 1}, 5: {'studi': 1, 'mathemat': 1, 'optim': 1, 'deliv': 1, 'method': 1, ',': 1, 'theori': 1, 'appl

`3 Calculate TermFrequency and generate a matrix`

Obliczam TermFrequency dla każdego słowa w zdaniu i generuje macierz dla całego dokumentu.

TF (t) = (Ile razy termin t występuje w dokumencie) / (Całkowita liczba terminów w zdaniu)

In [48]:
def _create_tf_matrix(freq_matrix):
    tf_matrix = {}

    for sent, f_table in freq_matrix.items():
        tf_table = {}

        count_words_in_sentence = len(f_table)
        for word, count in f_table.items():
            tf_table[word] = count / count_words_in_sentence

        tf_matrix[sent] = tf_table

    return tf_matrix

In [49]:
tf_matrix = _create_tf_matrix(freq_matrix)
print(tf_matrix)

{0: {'machin': 0.07142857142857142, 'learn': 0.07142857142857142, '(': 0.07142857142857142, 'ml': 0.07142857142857142, ')': 0.07142857142857142, 'studi': 0.07142857142857142, 'comput': 0.07142857142857142, 'algorithm': 0.07142857142857142, 'improv': 0.07142857142857142, 'automat': 0.07142857142857142, 'experi': 0.07142857142857142, 'use': 0.07142857142857142, 'data': 0.07142857142857142, '.': 0.07142857142857142}, 1: {'seen': 0.2, 'part': 0.2, 'artifici': 0.2, 'intellig': 0.2, '.': 0.2}, 2: {'machin': 0.047619047619047616, 'learn': 0.047619047619047616, 'algorithm': 0.047619047619047616, 'build': 0.047619047619047616, 'model': 0.047619047619047616, 'base': 0.047619047619047616, 'sampl': 0.047619047619047616, 'data': 0.09523809523809523, ',': 0.09523809523809523, 'known': 0.047619047619047616, '``': 0.047619047619047616, 'train': 0.047619047619047616, "''": 0.047619047619047616, 'order': 0.047619047619047616, 'make': 0.047619047619047616, 'predict': 0.047619047619047616, 'decis': 0.0476

`4 creating table for documents per words`

In [50]:
def _create_documents_per_words(freq_matrix):
    word_per_doc_table = {}

    for sent, f_table in freq_matrix.items():
        for word, count in f_table.items():
            if word in word_per_doc_table:
                word_per_doc_table[word] += 1
            else:
                word_per_doc_table[word] = 1

    return word_per_doc_table

In [51]:
count_doc_per_words = _create_documents_per_words(freq_matrix)
# print(count_doc_per_words)

{'machin': 85, 'learn': 138, '(': 50, 'ml': 5, ')': 50, 'studi': 7, 'comput': 28, 'algorithm': 62, 'improv': 10, 'automat': 1, 'experi': 5, 'use': 54, 'data': 63, '.': 261, 'seen': 2, 'part': 6, 'artifici': 19, 'intellig': 10, 'build': 4, 'model': 41, 'base': 8, 'sampl': 5, ',': 174, 'known': 11, '``': 20, 'train': 46, "''": 17, 'order': 6, 'make': 11, 'predict': 24, 'decis': 11, 'without': 8, 'explicitli': 3, 'program': 17, 'wide': 2, 'varieti': 4, 'applic': 10, 'email': 2, 'filter': 3, 'vision': 5, 'difficult': 2, 'unfeas': 1, 'develop': 5, 'convent': 2, 'perform': 26, 'need': 7, 'task': 17, 'subset': 4, 'close': 3, 'relat': 10, 'statist': 16, 'focus': 3, ';': 7, 'mathemat': 9, 'optim': 6, 'deliv': 3, 'method': 26, 'theori': 7, 'domain': 3, 'field': 17, 'mine': 10, 'exploratori': 1, 'analysi': 10, 'unsupervis': 15, 'across': 2, 'busi': 1, 'problem': 17, 'also': 15, 'refer': 3, 'analyt': 1, 'involv': 5, 'discov': 8, 'provid': 8, 'carri': 1, 'certain': 5, 'simpl': 1, 'assign': 3, 'poss

`5 Calculate IDF and generate a matrix`

Odwrotna częstotliwość dokumentu (IDF) określa, jak unikalne lub rzadkie jest słowo

In [52]:
def _create_idf_matrix(freq_matrix, count_doc_per_words, total_documents):
    idf_matrix = {}

    for sent, f_table in freq_matrix.items():
        idf_table = {}

        for word in f_table.keys():
            idf_table[word] = math.log10(total_documents / float(count_doc_per_words[word]))

        idf_matrix[sent] = idf_table

    return idf_matrix

In [53]:
idf_matrix = _create_idf_matrix(freq_matrix, count_doc_per_words, total_documents)
# print(idf_matrix)

{0: {'machin': 0.4905368227754652, 'learn': 0.28007666208852133, '(': 0.7209857441537391, 'ml': 1.720985744153739, ')': 0.7209857441537391, 'studi': 1.574857708475501, 'comput': 0.9727977171475386, 'algorithm': 0.627564058991504, 'improv': 1.4199557484897578, 'automat': 2.419955748489758, 'experi': 1.720985744153739, 'use': 0.6875619886667894, 'data': 0.6206151990361762, '.': 0.0033152411514769347}, 1: {'seen': 2.1189257528257768, 'part': 1.6418044981061142, 'artifici': 1.141202147536929, 'intellig': 1.4199557484897578, '.': 0.0033152411514769347}, 2: {'machin': 0.4905368227754652, 'learn': 0.28007666208852133, 'algorithm': 0.627564058991504, 'build': 1.8178957571617955, 'model': 0.8071718917700224, 'base': 1.5168657614978143, 'sampl': 1.720985744153739, 'data': 0.6206151990361762, ',': 0.17940650020715818, 'known': 1.378563063331533, '``': 1.1189257528257768, 'train': 0.7571979168081838, "''": 1.189506827111484, 'order': 1.6418044981061142, 'make': 1.378563063331533, 'predict': 1.0397

`6 Calculate TF-IDF and generate a matrix`

In [54]:
def _create_tf_idf_matrix(tf_matrix, idf_matrix):
    tf_idf_matrix = {}

    for (sent1, f_table1), (sent2, f_table2) in zip(tf_matrix.items(), idf_matrix.items()):

        tf_idf_table = {}

        for (word1, value1), (word2, value2) in zip(f_table1.items(), f_table2.items()):  # here, keys are the same in both the table
            tf_idf_table[word1] = float(value1 * value2)

        tf_idf_matrix[sent1] = tf_idf_table

    return tf_idf_matrix

In [55]:
tf_idf_matrix = _create_tf_idf_matrix(tf_matrix, idf_matrix)
# print(tf_idf_matrix)

{0: {'machin': 0.03503834448396179, 'learn': 0.020005475863465808, '(': 0.051498981725267075, 'ml': 0.12292755315383849, ')': 0.051498981725267075, 'studi': 0.11248983631967864, 'comput': 0.06948555122482418, 'algorithm': 0.044826004213678855, 'improv': 0.10142541060641126, 'automat': 0.1728539820349827, 'experi': 0.12292755315383849, 'use': 0.04911157061905638, 'data': 0.04432965707401259, '.': 0.0002368029393912096}, 1: {'seen': 0.42378515056515537, 'part': 0.32836089962122283, 'artifici': 0.2282404295073858, 'intellig': 0.2839911496979516, '.': 0.000663048230295387}, 2: {'machin': 0.0233588963226412, 'learn': 0.013336983908977206, 'algorithm': 0.029884002809119237, 'build': 0.08656646462675216, 'model': 0.03843675675095344, 'base': 0.07223170292846734, 'sampl': 0.081951702102559, 'data': 0.05910620943201678, ',': 0.017086333353062682, 'known': 0.06564586015864442, '``': 0.05328217870598937, 'train': 0.03605704365753256, "''": 0.05664318224340399, 'order': 0.07818116657648162, 'make'

`7 Important Algorithm: score the sentences`

In [56]:
def _score_sentences(tf_idf_matrix) -> dict:
    """
    score a sentence by its word's TF
    Basic algorithm: adding the TF frequency of every non-stop word in a sentence divided by total no of words in a sentence.
    :rtype: dict
    """

    sentenceValue = {}

    for sent, f_table in tf_idf_matrix.items():
        total_score_per_sentence = 0

        count_words_in_sentence = len(f_table)
        for word, score in f_table.items():
            total_score_per_sentence += score

        sentenceValue[sent] = total_score_per_sentence / count_words_in_sentence

    return sentenceValue


In [57]:
sentence_scores = _score_sentences(tf_idf_matrix)
# print(sentence_scores)

{0: 0.07133255036697676, 1: 0.25300813552440216, 2: 0.05305335853079526, 3: 0.06833973183463758, 4: 0.09278336785500811, 5: 0.08704545765674879, 6: 0.09954802301576178, 7: 0.10241721209452416, 8: 0.09775433271755679, 9: 0.12895190919543512, 10: 0.06597323419391442, 11: 0.13090097854018382, 12: 0.060679872664437674, 13: 0.13127400257774718, 14: 0.08486427124531028, 15: 0.09653490254550934, 16: 0.05464911493595631, 17: 0.09051731944812517, 18: 0.09741859907395185, 19: 0.07225364125810144, 20: 0.05777462998810356, 21: 0.0749482092717944, 22: 0.0837289108777769, 23: 0.09938912370568917, 24: 0.1399789579481274, 25: 0.03214738167899154, 26: 0.100881503626187, 27: 0.1650995253823934, 28: 0.0681193048418188, 29: 0.09309628540646776, 30: 0.09754744136553498, 31: 0.1289328612054436, 32: 0.10012101443986597, 33: 0.05762026561854745, 34: 0.07336318645385603, 35: 0.1428230555718743, 36: 0.13099313499403956, 37: 0.06778903635656487, 38: 0.14364483097478975, 39: 0.08949666312056201, 40: 0.09107090193

`8 Find the threshold`


In [58]:
def _find_average_score(sentenceValue) -> int:
    """
    Find the average score from the sentence value dictionary
    :rtype: int
    """
    sumValues = 0
    for entry in sentenceValue:
        sumValues += sentenceValue[entry]

    # Average value of a sentence from original summary_text
    average = (sumValues / len(sentenceValue))

    return average

In [60]:
threshold = _find_average_score(sentence_scores)
# print(threshold)

0.1073848423754357


`9 Important Algorithm: Generate the summary`


In [115]:
def _generate_summary(sentences, sentenceValue, threshold):
    # sentence_count = 0
    summary = ''

    for id, sentence in enumerate(sentences):
        if id in sentenceValue and sentenceValue[id] >= (threshold):
            # summary = summary + "\n["+ str(sentenceValue[id]) + "] " + sentence # pokarz score dla sentence
            summary += " " + sentence
            # sentence_count += 1

    summary = summary.strip()
    return summary

`Wywołanie wszystkich funkcji`

In [126]:
def run_summarization(text):
    sentences = sent_tokenize(text)
    total_documents = len(sentences)
    freq_matrix = _create_frequency_matrix(sentences)
    tf_matrix = _create_tf_matrix(freq_matrix)
    count_doc_per_words = _create_documents_per_words(freq_matrix)
    idf_matrix = _create_idf_matrix(freq_matrix, count_doc_per_words, total_documents)
    tf_idf_matrix = _create_tf_idf_matrix(tf_matrix, idf_matrix)
    sentence_scores = _score_sentences(tf_idf_matrix)
    threshold = _find_average_score(sentence_scores)
    summary = _generate_summary(sentences, sentence_scores, 1.9 * threshold)
    return summary 

In [127]:
summarization = run_summarization(text)
summarization

'It is seen as a part of artificial intelligence. A core objective of a learner is to generalize from its experience. The bias–variance decomposition is one way to quantify generalization error. There are two kinds of time complexity results. Other methods are based on estimated density and graph connectivity. It is a learning with no external rewards and no external teacher advice. The system is driven by the interaction between cognition and emotion. There is neither a separate reinforcement input nor an advice input from the environment. Classic examples include principal components analysis and cluster analysis. The method is strongly NP-hard and difficult to solve approximately. An artificial neuron that receives a signal can process it and then signal additional artificial neurons connected to it. The weight increases or decreases the strength of the signal at a connection. These decisions rely on objectivity and logical reasoning.'

In [125]:
print("format: ", type(summarization))
print("ilość znaków: ", len(summarization))
print("ilość słow: ", len(summarization.split(" ")))
print("ilość zdań: ", len(sent_tokenize(summarization)))

format:  <class 'str'>
ilość znaków:  950
ilość słow:  146
ilość zdań:  13
